# 生成死亡概率模型

In [1]:
ax_path = "./data/lc_ax_female.csv"
bx_path = "./data/lc_bx_female.csv"
kt_path = "./data/lc_kt_female.csv"
path = [ax_path, bx_path, kt_path]

import Prob 
death_Prob = Prob.D_Prob()
death_Prob.read_abk(path)

import ST
import numpy as np
from statistics import mean

np.random.seed(8)  # 设置固定的随机数种子


In [2]:
0.15200108460319872+ 0.8479989153968013

1.0

In [3]:
import pandas as pd
import os

lifespan = 75
x0 = 50
T0 = 2020
p = 1

terminal_live_cache_file = f'./data/terminal_live_cache_female_{x0}_{p}.csv'
interval_P_cache_file = f'./data/interval_P_cache_female_{x0}_{p}.csv'

# 尝试从CSV文件读取数据
if os.path.exists(terminal_live_cache_file):
    terminal_live_cache = pd.read_csv(terminal_live_cache_file, index_col=0).to_dict()['Value']
else:
    terminal_live_cache = {}

if os.path.exists(interval_P_cache_file):
    interval_P_cache_df = pd.read_csv(interval_P_cache_file)
    interval_P_cache = {(row['Start'], row['End']): row['Value'] for index, row in interval_P_cache_df.iterrows()}
else:
    interval_P_cache = {}

for i in range(0, (lifespan - x0) * p):
    if i == 0:
        terminal_live_cache[i] = death_Prob.accu_live(x0, T0, lifespan - x0, 0)
    else:
        terminal_live_cache[i] = death_Prob.interval_terminal_live(x0, T0, i, lifespan - x0, p)
    for j in range(i + 1, (lifespan - x0) * p + 1):
        if i == 0:
            interval_P_cache[(i, j)] = death_Prob.unit_P(x0, T0, j, p)
        else:
            interval_P_cache[(i, j)] = death_Prob.interval_death_P(x0, T0, i, j, p)

pd.DataFrame.from_dict(terminal_live_cache, orient='index', columns=['Value']).to_csv(terminal_live_cache_file)
interval_P_cache_df = pd.DataFrame([(key[0], key[1], value) for key, value in interval_P_cache.items()], columns=['Start', 'End', 'Value'])
interval_P_cache_df.to_csv(interval_P_cache_file, index=False)

In [4]:
def MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l):
    T = (lifespan - x0) * p - k
    N = buff_day * T
    sigma = 0.2

    # 你现有的MC函数代码继续
    result = []
    avg_res = []
    ave = []
    fee_result = []


    for _ in range(M):
        t_all, int_item = ST.intg_item(N, p, T, S0, r, l, g, sigma)
        intern_res = 0  # 内部的离散积分值，这里以年为单位
        int_death_payoff = 0
        for w in range(T):
            intern_res += int_item[int((w + 1) * buff_day) - 1] * interval_P_cache[(k, w+k+1)]
            # print("interval_P_cache",interval_P_cache[(k, w+k)])
            int_death_payoff += np.exp(-l*t_all[int((w+1)*buff_day) - 1])*interval_P_cache[(k, w+k+1)]
            # print("int_death_payoff", int_death_payoff)
        intern_res = intern_res + int_item[-1] * terminal_live_cache[k]
        # print("terminal_live_cache", terminal_live_cache[k])
        # print("np.exp(-l*T/p", np.exp(-l*T/p))
        fee = 1 - int_death_payoff - terminal_live_cache[k]*np.exp(-l*T/p)
        # print("fee",fee)

        result.append(intern_res)
        ave_j = np.mean(result)
        ave.append(ave_j)###用于画图，看结果稳不稳定
        fee_result.append(fee)

    avg_res = np.mean(result)
    fee_avg_res = np.mean(fee) 

    return avg_res, fee_avg_res, ave

In [5]:
def preliminary_search_per_i(initial_l, k, step_size, M, p, buff_day, S0, r, g, x0, T0, lifespan, max_iter=100):
    T = lifespan - x0 - k
    # l = 0.000129 * T**2 - 0.010775 * T + 0.200646
    l = initial_l
    lower_l, upper_l = None, None  # 初始化 lower_l 和 upper_l

    # 首先减小 l 直到 fee_avg_res - avg_res[-1] 变为负值
    for _ in range(max_iter):
        print("l: ", l)
        avg_res, fee_avg_res, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l)
        print("max(exp(g*t), S_t): ", avg_res)
        print("fee_avg_res: ", fee_avg_res)
        if fee_avg_res - avg_res < 0:
            upper_l = l + step_size  # 记录这个点的前一个点为 upper_l
            break
        l -= step_size

    # 从取负值的 l 开始，反转方向，以较小的步长增加 l
    step_size /= 3  # 减小步长
    for _ in range(max_iter):
        l += step_size
        print("l: ", l)
        avg_res, fee_avg_res, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l)
        print("max(exp(g*t), S_t): ", avg_res)
        print("fee_avg_res: ", fee_avg_res)
        if fee_avg_res - avg_res > 0:
            lower_l = l - step_size  # 记录这个点的前一个点为 lower_l
            upper_l = l
            break

    print("Before lower_l, upper_l:", lower_l, upper_l)
    # 第二轮搜索：缩小搜索范围
    new_lower_l = lower_l  # 初始化新的搜索范围
    new_upper_l = upper_l

    l_mid = (lower_l + upper_l) / 2  # 找到当前范围的中点
    print("l_mid: ", l_mid)
    avg_res_mid, fee_avg_res_mid, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l_mid)
    print("mid_max(exp(g*t), S_t): ", avg_res_mid)
    print("mid_fee_avg_res: ", fee_avg_res_mid)
    # 根据中点处的差值调整搜索范围
    if fee_avg_res_mid - avg_res_mid > 0:
        new_upper_l = l_mid
    else:
        new_lower_l = l_mid
        
    return new_lower_l, new_upper_l


In [6]:
def fine_search_per_i(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, lower_l, upper_l, fine_step_size):
    best_l = lower_l + fine_step_size #lower_l时diff<0,所以从lower_l+fine_step_size查看diff和0的区别
    min_difference = np.inf
    found_small_diff = False  # 标记是否找到小于0.0003的差异
    
    l = lower_l + fine_step_size
    while l <= upper_l - fine_step_size:
        print("l: ", l)
        avg_res, fee_avg_res, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l)
        print("max(exp(g*t), S_t): ", avg_res)
        print("fee_avg_res: ", fee_avg_res)
        difference = abs(fee_avg_res - avg_res)  # 计算差值的绝对值
        # 如果找到了小于0.0003的差异，标记为True
        if difference <0.000003:
            break
    
        if difference < 0.0003:
            found_small_diff = True
        
        # 在找到小于0.0003的差异后，如果差异又大于0.0025，结束搜索
        if found_small_diff and difference > 0.002:
            print(f"Ending search: difference increased beyond 0.0025 after finding a smaller difference.")
            break  # 结束循环
        
        # 更新最小差异和最优l
        if difference < min_difference:
            min_difference = difference
            best_l = l
            
        l += fine_step_size  # 使用非常小的步长
    
    print(f"Best l before stopping: {best_l}, Min difference before stopping: {min_difference}")
    return best_l, min_difference



In [7]:
# 初始化参数
M = 30000
x0 = 50
T0 = 2020
lifespan = 75

S0 = 1.0
p = 1
buff_p = int(12/p)
buff_day = 10*21*buff_p
r = 0.05
g = 0
tolerance = 1e-6 
step_size = 0.00006 #preliminary_search_per_i

k = 17   ####这次想要计算出来的年龄是多少的，k = 0,1,...,24
initial_l = 0.0104109

best_ls = []  # 用于存储每个索引i对应的最优l值
min_differences = []  # 用于存储每个索引i对应的最小差值

print("k:", k)
lower_l, upper_l = preliminary_search_per_i(initial_l, k, step_size, M, p, buff_day, S0, r, g, x0, T0, lifespan, max_iter=100)
print(f"索引{k}, new lower_l, upper_l: {lower_l}, {upper_l}")

if lower_l is not None and upper_l is not None:
    fine_step_size = 0.000001  # 精细搜索步长 
    best_l, min_difference = fine_search_per_i(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, lower_l, upper_l, fine_step_size)
    best_ls.append(best_l)
    min_differences.append(min_difference)
    print(f"Index {k}: Best l value: {best_l} with minimum difference: {min_difference}")
else:
    best_ls.append(None)
    min_differences.append(None)
    print(f"Index {k}: Unable to find suitable l interval during preliminary search.")

k: 17
l:  0.0104109
max(exp(g*t), S_t):  0.07801492536627527
fee_avg_res:  0.07839251517026613
l:  0.010350900000000001
max(exp(g*t), S_t):  0.07829648253593419
fee_avg_res:  0.07795902363674467
l:  0.0103709
max(exp(g*t), S_t):  0.07823028054695806
fee_avg_res:  0.0781035437421651
l:  0.0103909
max(exp(g*t), S_t):  0.07824918480647515
fee_avg_res:  0.07824804091878812
l:  0.010410899999999999
max(exp(g*t), S_t):  0.07892675394803478
fee_avg_res:  0.07839251517026613
l:  0.010430899999999998
max(exp(g*t), S_t):  0.07757853753925803
fee_avg_res:  0.07853696650025022
Before lower_l, upper_l: 0.010410899999999999 0.010430899999999998
l_mid:  0.010420899999999999
mid_max(exp(g*t), S_t):  0.07821735888763254
mid_fee_avg_res:  0.07846474370021661
索引17, new lower_l, upper_l: 0.010410899999999999, 0.010420899999999999
l:  0.010411899999999998
max(exp(g*t), S_t):  0.07704516637960238
fee_avg_res:  0.07839973828112035
l:  0.010412899999999998
max(exp(g*t), S_t):  0.0786273839540805
fee_avg_res: 